In [1]:
import os
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12500"
import torch
import torch.distributed as dist

torch.cuda.set_device(0)
dist.init_process_group(backend='nccl')
config_list = [
    "assets/HLLM/HLLM.yaml",
    "assets/overall/LLM_deepspeed.yaml"
]
from config.configurator import Config
cfg = Config(config_list)
print(cfg)
# torch.manual_seed(cfg.get("seed"))



/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.



General Hyper Parameters:
model = HLLM
seed = 2020
state = INFO
use_text = True
reproducibility = True
checkpoint_dir = saved
show_progress = True
log_wandb = False
data_path = /mnt/workspace/dataset/hllm/dataset
strategy = deepspeed
precision = bf16-mixed

Training Hyper Parameters:
epochs = 10
train_batch_size = 2
optim_args = {'learning_rate': 0.0001, 'weight_decay': 0.01}
eval_step = 1
stopping_step = 5

Evaluation Hyper Parameters:
eval_batch_size = 256
topk = [5, 10, 50, 200]
metrics = ['Recall', 'NDCG']
valid_metric = NDCG@200
metric_decimal_place = 7
eval_type = EvaluatorType.RANKING
valid_metric_bigger = True

Dataset Hyper Parameters:
MAX_ITEM_LIST_LENGTH = 50
MAX_TEXT_LENGTH = 64
text_keys = ['title', 'tag', 'description']
item_prompt = Compress the following sentence into embedding: 

Other Hyper Parameters: 
item_pretrain_dir = /mnt/workspace/model/TinyLlama-1.1B
item_llm_init = True
user_pretrain_dir = /mnt/workspace/model/TinyLlama-1.1B
user_llm_init = True
use_ft_flash

In [2]:
local_rank = 0
device = torch.device("cuda", local_rank)
cfg['device'] = device
extra_args_str = \
"""
--loss nce
--epochs 5
--dataset Pixel200K
--train_batch_size 16
--MAX_TEXT_LENGTH 256
--MAX_ITEM_LIST_LENGTH 10
--checkpoint_dir saved_path
--optim_args.learning_rate 1e-4
--item_pretrain_dir item_pretrain_dir
--user_pretrain_dir user_pretrain_dir
--text_path text_path
--text_keys [\"title\", \"tag\", \"description\"]
"""
extra_args_list = extra_args_str.strip().split("\n")
extra_args_list = [args.split(" ", maxsplit=1) for args in extra_args_list]
extra_args = []
for args_list in extra_args_list:
    extra_args.extend(args_list)
# cfg.parse_extra_args(extra_args)

from utils import init_seed
init_seed(cfg['seed'], cfg['reproducibility'])

# dist.destroy_process_group()

In [3]:
from data import load_data

dataload = load_data(cfg)

from data import bulid_dataloader

train_loader, valid_loader, test_loader = bulid_dataloader(cfg, dataload)

print(dataload)

2020
Pixel200K
The number of users: 200001
Average actions of users: 19.82828
The number of items: 96283
Average actions of items: 41.187927130720176
The number of inters: 3965656
The sparsity of the dataset: 99.9794063532928%


In [5]:
it = iter(train_loader)
first_bt = next(it)
print(first_bt)

/mnt/workspace/code/RecomAlgor_torch/data/dataset/collate_fn.py:41: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/mnt/workspace/code/RecomAlgor_torch/data/dataset/collate_fn.py:41: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/mnt/workspace/code/RecomAlgor_torch/data/dataset/collate_fn.py:41: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This shou

{'pos_item_ids': tensor([[54961, 71171, 20766, 80933, 12016, 60008, 31466, 27471, 53705, 87849,
         26910, 79237, 19517, 61800, 59291, 13761, 92493, 87832, 21196, 64375,
         81476, 70445, 62330,  5355, 58035,  6468, 45213, 28445, 15939, 69716,
         90774, 12262, 80347, 34469, 20374,  3443, 84371, 70657, 39371, 60806,
         31117, 40260, 10038, 49073, 10730, 59035,   254, 57752,  6383, 20356,
          8219],
        [47924, 29724, 78618,  3533, 52689, 87067, 16260, 36379, 67151, 43348,
         80382,  4177, 26698,  7122,  6055, 21510, 27899, 21784, 35018, 10500,
         11981, 33353, 30747, 12265,  7507,   683, 49077, 21027, 10717, 29542,
         40836,  6579, 28526,  4406, 35919, 10020, 23335, 11144, 54420, 68702,
         23400, 13725, 22471, 29782,  5140, 57207,  2910, 57112, 10475, 47002,
         40053]]), 'neg_item_ids': tensor([[65725, 84049, 82029, 60269, 32074, 67555,  5365, 80792, 68473, 81702,
         88669, 31532, 96191, 32186, 74597, 38574, 38829, 1679

In [7]:
import logging as log
from utils.logger import init_logger
init_logger(cfg)
logger = log.getLogger()

/usr/local/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
[rank0]:[W129 12:21:22.237742241 ProcessGroupNCCL.cpp:5072] Guessing device ID based on global rank. This can cause a hang if rank to GPU mapping is heterogeneous. You can specify device_id in init_process_group()


In [8]:
from utils import get_model
model = get_model(cfg['model'])(cfg, dataload)

29 Jan 12:21    INFO  create item llm
29 Jan 12:21    INFO  ******* create LLM /mnt/workspace/model/TinyLlama-1.1B *******
29 Jan 12:21    INFO  hf_config: LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "float32",
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.6",
  "use_cache": false,
  "vocab_size": 32001
}

29 Jan 12:21    INFO  xxxxx starting loading checkpoint
29 Jan 12:21    INFO  Using flash attention True for llama
29 Jan 12:21    INFO  Init True for llama


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. Fr

29 Jan 12:21    INFO  create user llm
29 Jan 12:21    INFO  ******* create LLM /mnt/workspace/model/TinyLlama-1.1B *******
29 Jan 12:21    INFO  hf_config: LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "float32",
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.6",
  "use_cache": false,
  "vocab_size": 32001
}

29 Jan 12:21    INFO  xxxxx starting loading checkpoint
29 Jan 12:21    INFO  Using flash attention True for llama
29 Jan 12:21    INFO  Init True for llama


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. Fr

29 Jan 12:21    INFO  nce thres setting to 0.99


In [10]:
from trainer.trainer import Trainer
world_size = torch.distributed.get_world_size()
trainer = Trainer(cfg, model)

ModuleNotFoundError: No module named 'lightning'